
# Text generation fundamentals

<sup>This notebook is a part of Natural Language Processing class at the University of Ljubljana, Faculty for computer and information science. Please contact [ales.zagar@fri.uni-lj.si](mailto:ales.zagar@fri.uni-lj.si) for any comments.</sub>

### **Introduction**

In this notebook, we will give a tour of the currently most prominent decoding methods, mainly *Greedy search*, *Beam search*, *Top-K sampling* and *Top-p sampling*. We will mostly work with the `.generate()` method, documented [here](https://huggingface.co/docs/transformers/main_classes/text_generation).


Let's import a tokenizer and a model. 

In [1]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
model = GPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

C:\Users\alesz\miniconda3\envs\nlp2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### **Greedy Search**

Greedy search simply selects the word with the highest probability as its next word: $w_t = argmax_{w}P(w | w_{1:t-1})$ at each timestep $t$. The following sketch shows greedy search. 

![Greedy Search](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/greedy_search.png)

Starting from the word $\text{"The"}$, the algorithm 
greedily chooses the next word of highest probability $\text{"nice"}$ and so on, so that the final generated word sequence is $\text{"The", "nice", "woman"}$ having an overall probability of $0.5 \times 0.4 = 0.2$.

In the following we will generate word sequences using GPT2 on the context $(\text{"I", "enjoy", "walking", "with", "my", "cute", "dog"})$. Let's see how greedy search can be used in `transformers` as follows:

In [2]:
# encode context the generation is conditioned on
input_ids = tokenizer.encode('I enjoy walking with my cute dog', return_tensors='pt')

# generate text until the output length (which includes the context length) reaches 50
greedy_output = model.generate(input_ids, max_length=50)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with my dog. I'm not sure if I'll ever be able to walk with my dog.

I'm not sure if I'll


The generated words following the context are reasonable, but the model quickly starts repeating itself! This is a very common problem in language generation in general and seems to be even more so in greedy and beam search - check out [Vijayakumar et al., 2016](https://arxiv.org/abs/1610.02424) and [Shao et al., 2017](https://arxiv.org/abs/1701.03185).

The major drawback of greedy search though is that it misses high probability words hidden behind a low probability word as can be seen in our sketch above:

The word $\text{"has"}$ with its high conditional probability of $0.9$ is hidden behind the word $\text{"dog"}$, which has only the second-highest conditional probability, so that greedy search misses the word sequence $\text{"The"}, \text{"dog"}, \text{"has"}$.

Thankfully, we have beam search to alleviate this problem!


### **Beam search**

Beam search reduces the risk of missing hidden high probability word sequences by keeping the most likely `num_beams` of hypotheses at each time step and eventually choosing the hypothesis that has the overall highest probability. Let's illustrate with `num_beams=2`:

![Beam search](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/beam_search.png)

At time step $1$, besides the most likely hypothesis $\text{"The", "nice"}$, beam search also keeps track of the second most likely one $\text{"The", "dog"}$. At time step $2$, beam search finds that the word sequence $\text{"The", "dog", "has"}$ has with $0.36$ a higher probability than $\text{"The", "nice", "woman"}$, which has $0.2$. Great, it has found the most likely word sequence in our toy example! 

Beam search will always find an output sequence with higher probability than greedy search, but is not guaranteed to find the most likely output. 

Let's see how beam search can be used in `transformers`. We set `num_beams > 1` and `early_stopping=True` so that generation is finished when all beam hypotheses reached the EOS token.

In [3]:
# activate beam search and early_stopping
beam_output = model.generate(
    input_ids,  
    max_length=50, 
    num_beams=5, 
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I'm not sure if I'll ever be able to walk with him again. I'm not sure if I'll


While the result is arguably more fluent, the output still includes repetitions of the same word sequences.  
A simple remedy is to introduce *n-grams* (*a.k.a* word sequences of $n$ words) penalties as introduced by [Paulus et al. (2017)](https://arxiv.org/abs/1705.04304) and [Klein et al. (2017)](https://arxiv.org/abs/1701.02810). The most common *n-grams* penalty makes sure that no *n-gram* appears twice by manually setting the probability of next words that could create an already seen *n-gram* to $0$.

Let's try it out by setting `no_repeat_ngram_size=2` so that no *2-gram* appears twice:

In [4]:
# set no_repeat_ngram_size to 2
beam_output = model.generate(
    input_ids, 
    max_length=50, 
    num_beams=5, 
    no_repeat_ngram_size=2, 
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time for me to take a break


Nice, that looks much better! We can see that the repetition does not appear anymore. Nevertheless, *n-gram* penalties have to be used with care. An article generated about the city *New York* should not use a *2-gram* penalty or otherwise, the name of the city would only appear once in the whole text!

Another important feature about beam search is that we can compare the top beams after generation and choose the generated beam that fits our purpose best. 

In `transformers`, we simply set the parameter `num_return_sequences` to the number of highest scoring beams that should be returned. Make sure though that `num_return_sequences <= num_beams`!

In [5]:
# set return_num_sequences > 1
beam_outputs = model.generate(
    input_ids, 
    max_length=50, 
    num_beams=5, 
    no_repeat_ngram_size=2, 
    num_return_sequences=5, 
    early_stopping=True
)

# now we have 3 output sequences
print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time for me to take a break
1: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time for me to get back to
2: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with her again.

I've been thinking about this for a while now, and I think it's time for me to take a break
3: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with her again.

I've been thinking about this for a while now, and I think it's time for me to get back to
4: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about 

As can be seen, the five beam hypotheses are only marginally different to each other - which should not be too surprising when using only 5 beams.

In open-ended generation, a couple of reasons have recently been brought forward why beam search might not be the best possible option:

- Beam search can work very well in tasks where the length of the desired generation is more or less predictable as in machine translation or summarization - see [Murray et al. (2018)](https://arxiv.org/abs/1808.10006) and [Yang et al. (2018)](https://arxiv.org/abs/1808.09582). But this is not the case for open-ended generation where the desired output length can vary greatly, e.g. dialog and story generation.

- We have seen that beam search heavily suffers from repetitive generation. This is especially hard to control with *n-gram*- or other penalties in story generation since finding a good trade-off between forced "no-repetition" and repeating cycles of identical *n-grams* requires a lot of finetuning.

- As argued in [Ari Holtzman et al. (2019)](https://arxiv.org/abs/1904.09751), high quality human language does not follow a distribution of high probability next words. In other words, as humans, we want generated text to surprise us and not to be boring/predictable. The authors show this nicely by plotting the probability, a model would give to human text vs. what beam search does.

![alt text](https://blog.fastforwardlabs.com/images/2019/05/Screen_Shot_2019_05_08_at_3_06_36_PM-1557342561886.png)


### **Sampling**

In its most basic form, sampling means randomly picking the next word $w_t$ according to its conditional probability distribution:

$$w_t \sim P(w|w_{1:t-1})$$

Taking the example from above, the following graphic visualizes language generation when sampling.

![vanilla_sampling](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/sampling_search.png)

It becomes obvious that language generation using sampling is not *deterministic* anymore. The word 
$\text{"car"}$ is sampled from the conditioned probability distribution $P(w | \text{"The"})$, followed by sampling $\text{"drives"}$ from $P(w | \text{"The"}, \text{"car"})$.

In `transformers`, we set `do_sample=True` and deactivate *Top-K* sampling (more on this later) via `top_k=0`. In the following, we will fix `random_seed=0` for illustration purposes. Feel free to change the `random_seed` to play around with the model.


In [6]:
# Set a random seed
torch.manual_seed(42)

# activate sampling and deactivate top_k by setting top_k sampling to 0
sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=50, 
    top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog but what I love about being a dog cat person is being a pet being with people who can treat you. I feel happy to be such a pet person and get to meet so many people. I don't think


Interesting! The text seems alright - but when taking a closer look, it is not very coherent and don't sound like they were written by a human. That is the big problem when sampling word sequences: The models often generate incoherent gibberish, *cf.* [Ari Holtzman et al. (2019)](https://arxiv.org/abs/1904.09751).

A trick is to make the distribution $P(w|w_{1:t-1})$ sharper (increasing the likelihood of high probability words and decreasing the likelihood of low probability words) by lowering the so-called `temperature` of the [softmax](https://en.wikipedia.org/wiki/Softmax_function#Smooth_arg_max). Check out the temperature demonstration on https://lukesalamone.github.io/posts/what-is-temperature/

An illustration of applying temperature to our example from above could look as follows.

![top_p_sampling](https://github.com/patrickvonplaten/scientific_images/blob/master/sampling_search_with_temp.png?raw=true)

The conditional next word distribution of step $t=1$ becomes much sharper leaving almost no chance for word $\text{"car"}$ to be selected.


Let's see how we can cool down the distribution in the library by setting `temperature=0.7`:

In [7]:
# set seed to reproduce results. Feel free to change the seed though to get different results
import torch

# Set a random seed
torch.manual_seed(42)

# use temperature to decrease the sensitivity to low probability candidates
sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=50, 
    top_k=0, 
    temperature=0.7
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog but I also love people being nice. I love being in a party with a group of friends and having the opportunity to see people who are so different from me.

I'm not sure if I'll be


OK. There are less weird n-grams and the output is a bit more coherent now! While applying temperature can make a distribution less random, in its limit, when setting `temperature` $ \to 0$, temperature scaled sampling becomes equal to greedy decoding and will suffer from the same problems as before. 



### **Top-K Sampling**

[Fan et. al (2018)](https://arxiv.org/pdf/1805.04833.pdf) introduced a simple, but very powerful sampling scheme, called ***Top-K*** sampling. In *Top-K* sampling, the *K* most likely next words are filtered and the probability mass is redistributed among only those *K* next words. 
GPT2 adopted this sampling scheme, which was one of the reasons for its success in story generation. 

We extend the range of words used for both sampling steps in the example above from 3 words to 10 words to better illustrate *Top-K* sampling.

![top_k_sampling](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/top_k_sampling.png)

Having set $K = 6$, in both sampling steps we limit our sampling pool to 6 words. While the 6 most likely words, defined as $V_{\text{top-K}}$ encompass only *ca.* two-thirds of the whole probability mass in the first step, it includes almost all of the probability mass in the second step. Nevertheless, we see that it successfully eliminates the rather weird candidates $\text{"not", "the", "small", "told"}$ 
in the second sampling step.


Let's see how *Top-K* can be used in the library by setting `top_k=10`:

In [8]:
# Set a random seed
torch.manual_seed(42)

# set top_k to 50
sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=50, 
    top_k=10
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog but sometimes I get nervous when she is around. I don't like being alone in the house, so I usually just sit down and watch her play with my phone."

A spokesman for the company said:


One concern though with *Top-K* sampling is that it does not dynamically adapt the number of words that are filtered from the next word probability distribution $P(w|w_{1:t-1})$.
This can be problematic as some words might be sampled from a very sharp distribution (distribution on the right in the graph above), whereas others from a much more flat distribution (distribution on the left in the graph above). 

In step $t=1$, *Top-K* eliminates the possibility to 
sample $\text{"people", "big", "house", "cat"}$, which seem like reasonable candidates. On the other hand, in step $t=2$ the method includes the arguably ill-fitted words $\text{"down", "a"}$ in the sample pool of words. Thus, limiting the sample pool to a fixed size *K* could endanger the model to produce gibberish for sharp distributions and limit the model's creativity for flat distribution.
This intuition led [Ari Holtzman et al. (2019)](https://arxiv.org/abs/1904.09751) to create ***Top-p***- or ***nucleus***-sampling. 



### **Top-p (nucleus) sampling**

Instead of sampling only from the most likely *K* words, in *Top-p* sampling chooses from the smallest possible set of words whose cumulative probability exceeds the probability *p*. The probability mass is then redistributed among this set of words. This way, the size of the set of words (*a.k.a* the number of words in the set) can dynamically increase and decrease according to the next word's probability distribution. Ok, that was very wordy, let's visualize.

![top_p_sampling](https://github.com/patrickvonplaten/scientific_images/blob/master/top_p_sampling.png?raw=true)

Having set $p=0.92$, *Top-p* sampling picks the *minimum* number of words to exceed together $p=92\%$ of the probability mass, defined as $V_{\text{top-p}}$. In the first example, this included the 9 most likely words, whereas it only has to pick the top 3 words in the second example to exceed 92%. Quite simple actually! It can be seen that it keeps a wide range of words where the next word is arguably less predictable, *e.g.* $P(w | \text{"The"})$, and only a few words when the next word seems more predictable, *e.g.* $P(w | \text{"The", "car"})$.

Alright, time to check it out in `transformers`!
We activate *Top-p* sampling by setting `0 < top_p < 1`:

In [9]:
# Set a random seed
torch.manual_seed(42)

# deactivate top_k sampling and sample only from 92% most likely words
sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=50, 
    top_p=0.92, 
    top_k=0  # We need to deactivate top_k if we want to use only top_p!
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog but what I love about being a dog cat person is being a pet being with people who can treat you. I feel happy to be such a pet person and get to meet so many people. I don't think


While in theory, *Top-p* seems more elegant than *Top-K*, both methods work  well in practice. *Top-p* can also be used in combination with *Top-K*, which can avoid very low ranked words while allowing for some dynamic selection.

Finally, to get multiple independently sampled outputs, we can *again* set the parameter `num_return_sequences > 1`: 

In [10]:
# Set a random seed
torch.manual_seed(42)

# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
sample_outputs = model.generate(
    input_ids,
    do_sample=True, 
    max_length=50, 
    top_k=50, 
    top_p=0.95, 
    num_return_sequences=3
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: I enjoy walking with my cute dog but sometimes I get nervous when she is around. I've been told that with her alone, she will usually wander off and then try to chase me. It's nice to know that I have this control over her
1: I enjoy walking with my cute dog. I think she is the same one I like to walk with my dog, I think she is about as girly as my first dog. I hope we can find an apartment for her when we come back.
2: I enjoy walking with my cute dog, but there's so much to say about him that I am going to miss it all. He has been so supportive and even had my number in his bag.

I hope I can say the following to


Find below a list of the most popular decoding methods and how to use them using `.generate()` method.

- **Greedy Decoding**: Invoked by calling `_greedy_search()` when `num_beams=1` and `do_sample=False`. This method selects the word with the highest probability at each step.

- **Contrastive Search**: Triggered by `_contrastive_search()` if `penalty_alpha>0` and `top_k>1`. This technique modifies the selection process to diversify the output.

- **Multinomial Sampling**: Utilizes `_sample()` when `num_beams=1` and `do_sample=True`, introducing randomness by sampling from the distribution of possible next words.

- **Beam-Search Decoding**: Engaged through `_beam_search()` when `num_beams>1` and `do_sample=False`, considering multiple best paths simultaneously to enhance coherency.

- **Beam-Search Multinomial Sampling**: Invoked by `_beam_sample()` if `num_beams>1` and `do_sample=True`, combining beam search's thoroughness with the randomness of sampling.

- **Diverse Beam-Search Decoding**: Employed via `_group_beam_search()`, when `num_beams>1` and `num_beam_groups>1`, to ensure variety among the considered paths.

- **Constrained Beam-Search Decoding**: Activated by `_constrained_beam_search()`, if `constraints!=None` or `force_words_ids!=None`, imposing specific constraints on the search process.

- **Assisted Decoding**: Utilized by `_assisted_decoding()`, if `assistant_model` or `prompt_lookup_num_tokens` is passed to `.generate()`, leveraging external models or prompts to guide the generation.


# Perplexity
Perplexity is a metric commonly used to evaluate the performance of language models in natural language processing (NLP). It measures how well a language model predicts a sample and is a function of the probability that the model assigns to a sequence of words. In essence, perplexity quantifies the uncertainty of a language model in predicting (or "guessing") the next token in a sequence.

### Definition
The perplexity (P) of a language model for a given sequence of words $W = w_1, w_2, ..., w_N$ is defined as the exponential of the entropy (or cross-entropy if comparing against a true distribution) of the sequence, given by:

$$ P(W) = 2^{H(W)} $$

where $H(W)$ is the entropy of the sequence $W$, calculated as:

$$ H(W) = -\frac{1}{N} \sum_{i=1}^{N} \log_2 p(w_i | w_{i-1}, ..., w_1) $$

Here, $p(w_i | w_{i-1}, ..., w_1)$ represents the conditional probability of word $w_i$ given all the preceding words $w_1, ..., w_{i-1}$, as predicted by the language model, and $N$ is the length of the sequence.

### Interpretation
- **Lower Perplexity:** Indicates a better-performing model, as it means the model has less uncertainty in predicting the next word in a sequence. A lower perplexity score suggests that the language model is more confident and accurate in its predictions.
- **Higher Perplexity:** Suggests a model with greater uncertainty and less ability to predict the next word accurately. High perplexity indicates that the model's predictions are more spread out over a larger set of possible words.


In [11]:
import torch
from tqdm import tqdm

texts = ['I am a very hard worker', 
         'worker hard very a am I', 
         'I like ice cream', 
         'I like icecream', 
         'I like driving cars', 
         'I lik driving cars']  # List of texts to process

for t in texts:  
    encodings = tokenizer(t, return_tensors="pt")  
    
    max_length = model.config.n_positions  # Maximum sequence length the model can handle
    stride = 512  # The step size for breaking the text into overlapping segments if needed
    seq_len = encodings.input_ids.size(1)  # The length of the tokenized sequence
    
    nlls = []  # A list to store the negative log-likelihoods for each segment
    prev_end_loc = 0  # Variable to keep track of the end location of the previous segment
    
    for begin_loc in range(0, seq_len, stride):  # Loop over the sequence in strides to handle long texts
        end_loc = min(begin_loc + max_length, seq_len)  # Determine the end location of the current segment
        trg_len = end_loc - prev_end_loc  # The target length may differ from stride in the last loop
        
        input_ids = encodings.input_ids[:, begin_loc:end_loc]  # Extract the input IDs for the current segment
        target_ids = input_ids.clone()  # Clone input_ids to create target_ids for calculating loss
        target_ids[:, :-trg_len] = -100  # Mask tokens not part of the target length with -100 (ignored by loss function)
    
        with torch.no_grad():  # Disable gradient calculations
            outputs = model(input_ids, labels=target_ids)  # Forward pass through the model with masked target IDs
    
            # Loss is calculated using CrossEntropyLoss which averages over valid (non-masked) labels
            # Note: The model calculates loss over trg_len - 1 labels due to an internal shift to the left by 1
            neg_log_likelihood = outputs.loss  # Extract the negative log-likelihood from the model outputs
    
        nlls.append(neg_log_likelihood)  # Append the NLL of the current segment to the list
    
        prev_end_loc = end_loc  # Update the end location for the next iteration
        if end_loc == seq_len:  # Break the loop if the end of the sequence is reached
            break
    
    ppl = torch.exp(torch.stack(nlls).mean())  # Calculate perplexity as the exponential of the average NLL
    print(ppl.numpy(), t)  # Print the calculated perplexity value


44.10664 I am a very hard worker
5751.8135 worker hard very a am I
187.38785 I like ice cream
779.6343 I like icecream
809.519 I like driving cars
185511.25 I lik driving cars


As we can see, the perplexity score for the inversed order of words is much higher. Also, sequences with misspelled words are penalized. 

# References

Parts of this notebook were taken from [notebook](https://colab.research.google.com/github/huggingface/blog/blob/master/notebooks/02_how_to_generate.ipynb).